In [7]:
import pandas as pd
import jieba
import csv
import gensim
from pprint import pprint

data = pd.read_csv(r"data_topic.xlsx",encoding="gbk")
content = data["评论内容"].str.replace(r"[0-9]","",regex=True)

sentence_cut = [jieba.lcut(line) for line in content]
stopwords = pd.read_csv(r"datas\停用词表\哈工大停用词表.txt",header=None,quoting = csv.QUOTE_NONE,delimiter="\t")
stopwords = stopwords[0].tolist()
new_stopwords = pd.read_csv(r"datas\停用词表\中文停用词库.txt",header=None)
stopwords.extend(new_stopwords[0].tolist())
#模型构建
not_stopwords = [[word for word in line if word not in stopwords and len(word)>1]for line in sentence_cut]
#送进模型里获得词向量
model =gensim.models.Word2Vec(not_stopwords,min_count=185,window=5) 
#获得所有词汇
words = list(model.wv.key_to_index)

UnicodeDecodeError: 'gbk' codec can't decode byte 0x8b in position 123: illegal multibyte sequence

In [8]:
#获得词频
from collections import defaultdict
frequency =defaultdict(int)
for line in not_stopwords:
    for token in line:
        frequency[token] +=1
words_frequency = [frequency[word] for word in words]
word_frequency = pd.DataFrame({"单词":words,"频率":words_frequency})
word_frequency

NameError: name 'not_stopwords' is not defined

In [ ]:
#分别用PCA和t-SNE降维看效果
from sklearn.decomposition import PCA
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
#先看PCA的可视化效果
X = model.wv[words]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
ig, ax = plt.subplots(figsize=(20, 10))
# 绘制散点图，圆圈就是出现的频率
ax.scatter(result[:, 0], result[:, 1], c='SeaGreen', s=words_frequency,alpha=0.5)
# 不同的圆圈有时会重叠在一起，可以使用adjustText修正文字重叠现象
from adjustText import adjust_text
new_texts = [plt.text(x, y, text, fontsize=12) for x, y, text in zip(result[:, 0], result[:, 1], words)]
adjust_text(new_texts, 
            only_move={'text': 'x'},
            arrowprops=dict(arrowstyle='-', color='grey'))
# 美观起见隐藏顶部与右侧边框线
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
from sklearn.manifold import TSNE
ts =TSNE(2)
result2 = ts.fit_transform(X)
fig, ax = plt.subplots(figsize=(20, 10))
# 绘制散点图
ax.scatter(result2[:, 0], result2[:, 1], c='SeaGreen', s=words_frequency,alpha=0.5)
# 使用adjustText修正文字重叠现象
new_texts = [plt.text(x, y, text, fontsize=12) for x, y, text in zip(result2[:, 0], result2[:, 1], words)]
adjust_text(new_texts, 
            only_move={'text': 'x'},
            arrowprops=dict(arrowstyle='-', color='grey'))
# 美观起见隐藏顶部与右侧边框线
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
#进行聚类
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples

#用轮廓系数绘制学习曲线找出最优类别数
ss = []
krange = list(range(2,7))
for k in range(2,7):
    model = KMeans(n_clusters= k,random_state=0)
    model=model.fit(result)
    ss.append(silhouette_score(result,model.labels_))
plt.plot(krange,ss)

In [ ]:
#用PCA降维数据后的分类结果
fig, ax = plt.subplots(figsize=(15, 15))
from sklearn.cluster import KMeans
model = KMeans(n_clusters= 3,random_state=0,init="k-means++")
model = model.fit(result2)              
# 将聚类的结果和中心点的结果都画在原图里面
plt.scatter(result2[:, 0], result2[:, 1], c = model.labels_,s= words_frequency,alpha=0.5)
plt.scatter(model.cluster_centers_[: , 0],model.cluster_centers_[:, 1], color = "red") #加上每一个的中心带你

new_texts = [plt.text(x, y, text, fontsize=12) for x, y, text in zip(result2[:, 0], result2[:, 1], words)]
adjust_text(new_texts, 
            only_move={'text': 'x'},
            arrowprops=dict(arrowstyle='-', color='grey'))

# 美观起见隐藏顶部与右侧边框线
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 定义KMeans，以及K值
kmeans = KMeans(n_clusters=3)
# 根据数据data进行聚类，结果存放于result_list中
result_list = kmeans.fit_predict(result)
# 将原始的数据data和聚类结果result_list
# 传入对应的函数计算出该结果下的轮廓系数
score = silhouette_score(data, result_list)